In [2]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [11]:
data = np.loadtxt("optimal_solution.txt").T

time = data[0]
altitude = data[1]
speed = data[2]
mass = data[3]
thrust = data[4]

In [12]:
fig = make_subplots(rows=4, cols=1,
                    subplot_titles=("Altitude", "Speed", "Thrust", "Mass"))

fig.add_trace(
    go.Scatter(x=time, y=altitude),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=time, y=speed),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=time, y=thrust),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=time, y=mass),
    row=4, col=1
)

fig.update_layout(height=600, width=800, title_text="PolyMPC collocation_fix_jw branch, poly=10, segment=1")
fig.update(layout_showlegend=False)
fig.show()